Different algorithms were used to estimate both the price and the street where the house is located.
The KNeighborsClassifier model was used because it was the most reliable in predictions. The model worked with a probability of 0.81 for house price prediction and 0.85 for street prediction. The code lines and outputs are as follows.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [48]:
df = pd.read_csv('datasets/NY_House_Dataset.csv')
df.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [49]:
df['SUBLOCALITY'].unique()

array(['Manhattan', 'New York County', 'Richmond County', 'Kings County',
       'New York', 'East Bronx', 'Brooklyn', 'The Bronx', 'Queens',
       'Staten Island', 'Queens County', 'Bronx County', 'Coney Island',
       'Brooklyn Heights', 'Jackson Heights', 'Riverdale', 'Rego Park',
       'Fort Hamilton', 'Flushing', 'Dumbo', 'Snyder Avenue'],
      dtype=object)

In [50]:
df = df[['TYPE','BEDS','BATH','LOCALITY','SUBLOCALITY','STREET_NAME','PROPERTYSQFT','PRICE']]

In [51]:
df.head()

,TYPE,BEDS,BATH,LOCALITY,SUBLOCALITY,STREET_NAME,PROPERTYSQFT,PRICE
0,Condo for sale,2,2.000000,New York,Manhattan,East 55th Street,1400.0,315000
1,Condo for sale,7,10.000000,New York,New York County,New York,17545.0,195000000
2,House for sale,4,2.000000,New York,Richmond County,Staten Island,2015.0,260000
3,Condo for sale,3,1.000000,New York,New York County,New York,445.0,69000
4,Townhouse for sale,7,2.373861,New York,New York County,New York,14175.0,55000000


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4801 entries, 0 to 4800
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TYPE          4801 non-null   object 
 1   BEDS          4801 non-null   int64  
 2   BATH          4801 non-null   float64
 3   LOCALITY      4801 non-null   object 
 4   SUBLOCALITY   4801 non-null   object 
 5   STREET_NAME   4801 non-null   object 
 6   PROPERTYSQFT  4801 non-null   float64
 7   PRICE         4801 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 300.2+ KB


In [53]:
df['STREET_NAME'].nunique()

174

In [54]:
df.columns = ['type','beds','bath','location','sublocation','street','property','price']

In [77]:
df.drop('pred',axis=1,inplace=True)

In [78]:
df['bath'] = df['bath'].astype(int)
df['property'] = df['property'].astype(int)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4801 entries, 0 to 4800
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   type         4801 non-null   int32
 1   beds         4801 non-null   int64
 2   bath         4801 non-null   int32
 3   location     4801 non-null   int32
 4   sublocation  4801 non-null   int32
 5   property     4801 non-null   int32
 6   price        4801 non-null   int64
dtypes: int32(5), int64(2)
memory usage: 168.9 KB


In [80]:
cat_cols = [col for col in df.columns if df[col].dtype in ['object']]
cat_cols

[]

In [59]:
for i in cat_cols:
    print(df[i].unique())

['Condo for sale' 'House for sale' 'Townhouse for sale' 'Co-op for sale'
 'Multi-family home for sale' 'For sale' 'Contingent' 'Land for sale'
 'Foreclosure' 'Pending' 'Coming Soon' 'Mobile house for sale'
 'Condop for sale']
['New York' 'New York County' 'The Bronx' 'Kings County' 'Bronx County'
 'Queens County' 'Richmond County' 'United States' 'Brooklyn' 'Queens'
 'Flatbush']
['Manhattan' 'New York County' 'Richmond County' 'Kings County' 'New York'
 'East Bronx' 'Brooklyn' 'The Bronx' 'Queens' 'Staten Island'
 'Queens County' 'Bronx County' 'Coney Island' 'Brooklyn Heights'
 'Jackson Heights' 'Riverdale' 'Rego Park' 'Fort Hamilton' 'Flushing'
 'Dumbo' 'Snyder Avenue']
['East 55th Street' 'New York' 'Staten Island' 'Brooklyn' 'Manhattan'
 'Morrison Avenue' 'Midwood' 'Concourse Village' 'Flushing' 'Elmhurst'
 'Annadale' 'Queens' 'Fort Hamilton' 'North Riverdale' 'Rego Park'
 'Forest Hills' 'The Bronx' 'Dongan Hills' 'Jackson Heights'
 'Kings County' 'Clifton' 'Mariners Harbor' 'Dyker

In [81]:
lb = LabelEncoder()

for col in cat_cols:
    df[col] = lb.fit_transform(df[col])

In [82]:
df.head()

,type,beds,bath,location,sublocation,property,price
0,2,2,2,4,10,1400,315000
1,2,7,10,4,12,17545,195000000
2,7,4,2,4,16,2015,260000
3,2,3,1,4,12,445,69000
4,12,7,2,4,12,14175,55000000


In [71]:
df.drop('street',axis=1,inplace=True)

In [72]:
x = df.drop('price',axis=1)
y = df['price']

In [85]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [86]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train)
knn_pred = knn.predict(x)
print(mean_squared_error(y,knn_pred))
print(r2_score(y,knn_pred))
print(accuracy_score(y,knn_pred))

4927881340595.631
0.9949866239915899
0.5730056238283691


In [66]:
linear = LinearRegression()
linear.fit(x_train,y_train)
linear_pred = linear.predict(x)
print(mean_squared_error(y,linear_pred))
print(r2_score(y,linear_pred))

969106926436160.5
0.014079869465394967


In [67]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)
rf_y_pred = rf_model.predict(x)
print(mean_squared_error(y,rf_y_pred))
print(r2_score(y,rf_y_pred))

237616891275591.56
0.7582606520777037


In [88]:
with open ('price.pickle','wb') as file:
    pickle.dump(knn, file)

In [68]:
df['pred'] = knn_pred